<a href="https://colab.research.google.com/github/marioluciofjr/Livro_com_IA_generativa/blob/main/gerarLivro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação das dependências necessárias
!pip install -q google-generativeai beautifulsoup4 requests markdown

In [ ]:
# Importações necessárias
import os
import time
import textwrap
import requests
import google.generativeai as genai
from bs4 import BeautifulSoup
import markdown
from IPython.display import Markdown, display
from google.colab import userdata

In [ ]:
# Variáveis de entrada para gerar o livro
tema = input("Digite o tema do livro: ")
num_capitulos = int(input("Digite o número de capítulos: "))
tom = input("Digite o tom de voz do livro: ")
contexto = input("Digite o contexto e demais detalhes do livro: ")
url_referencia = input("Digite um URL de referência (opcional, pressione Enter para pular): ")


# Configuração da API do Gemini
def configurar_api():
    """
    Configura a API do Google Gemini.
    Você precisará ter uma chave de API do Google AI Studio.
    """
    try:
        # Método 1: Usar chave armazenada no Google Colab
        API_KEY = userdata.get('sua_api')

        # Método 2: Alternativa - inserir a chave diretamente
        # API_KEY = "sua_chave_api_aqui"  # Descomente esta linha se preferir inserir diretamente

        # Configura a API
        genai.configure(api_key=API_KEY)

        print("✅ API do Gemini configurada com sucesso!")
        return True
    except Exception as e:
        print(f"❌ Erro ao configurar a API: {str(e)}")
        print("Por favor, verifique sua chave de API.")
        return False

# Função para criar um modelo Gemini
def criar_modelo():
    """
    Cria e retorna uma instância do modelo Gemini escolhido.
    """
    try:
        modelo = genai.GenerativeModel('gemini-2.0-flash-thinking-exp-01-21')
        print("✅ Modelo Gemini carregado com sucesso!")
        return modelo
    except Exception as e:
        print(f"❌ Erro ao carregar o modelo: {str(e)}")
        return None

# Função para extrair conteúdo de uma URL
def extrair_conteudo_url(url):
    """
    Extrai o conteúdo textual de uma URL e converte para markdown.

    Args:
        url: URL da página web

    Returns:
        Conteúdo da página em formato markdown
    """
    if not url or url.strip() == "":
        return ""

    try:
        print(f"🌐 Extraindo conteúdo de {url}...")
        response = requests.get(url)
        response.raise_for_status()  # Verifica se houve erro na requisição

        # Parseia o HTML
        soup = BeautifulSoup(response.text, 'html.parser')

        # Remove scripts, estilos e outros elementos não desejados
        for element in soup(['script', 'style', 'header', 'footer', 'nav']):
            element.decompose()

        # Extrai o texto principal
        texto = soup.get_text()

        # Limpa o texto (remove linhas em branco extras, etc)
        linhas = [linha.strip() for linha in texto.split('\n') if linha.strip()]
        texto_limpo = '\n\n'.join(linhas)

        # Converte para markdown para facilitar comunicação com o LLM
        texto_markdown = texto_limpo  # Versão simplificada, mantém o texto como está

        print(f"✅ Conteúdo extraído com sucesso! ({len(texto_markdown)} caracteres)")
        return texto_markdown
    except Exception as e:
        print(f"❌ Erro ao extrair conteúdo da URL: {str(e)}")
        return ""

# Classe base para os agentes
class Agente:
    """
    Classe base para os agentes de criação de conteúdo.
    """
    def __init__(self, modelo, nome, role, goal, backstory, temperatura=0.7, max_tokens=9000):
        """
        Inicializa um agente.

        Args:
            modelo: Instância do modelo Gemini
            nome: Nome do agente
            role: Papel do agente
            goal: Objetivo do agente
            backstory: História de fundo do agente
            temperatura: Nível de criatividade do modelo (0.0 a 1.0)
            max_tokens: Limite máximo de tokens para respostas
        """
        self.modelo = modelo
        self.nome = nome
        self.role = role
        self.goal = goal
        self.backstory = backstory
        self.temperatura = temperatura
        self.max_tokens = max_tokens

    def _gerar_resposta(self, prompt):
        """
        Gera uma resposta do modelo com base no prompt fornecido.

        Args:
            prompt: Texto do prompt para o modelo

        Returns:
            Texto da resposta gerada
        """
        try:
            resposta = self.modelo.generate_content(
                prompt,
                generation_config=genai.GenerationConfig(
                    temperature=self.temperatura,
                    max_output_tokens=self.max_tokens,
                    top_p=0.95,
                    top_k=40
                )
            )

            return resposta.text
        except Exception as e:
            print(f"❌ Erro ao gerar conteúdo com o {self.nome}: {str(e)}")
            return None

    def _formatar_saida(self, texto):
        """
        Formata o texto de saída para melhor legibilidade.

        Args:
            texto: Texto a ser formatado

        Returns:
            Texto formatado
        """
        texto = texto.strip()
        return texto


class AgenteAutor(Agente):
    """
    Agente responsável por criar a primeira versão do conteúdo do livro.
    """
    def __init__(self, modelo):
        super().__init__(
            modelo,
            "Agente Autor",
            "Escritor Criativo",
            "Criar a primeira versão do conteúdo do livro com originalidade, coesão e fluidez narrativa.",
            "Você é um autor apaixonado por contar histórias que cativam. Com vasta experiência em desenvolver enredos, personagens e diálogos envolventes, sua missão é dar vida ao esboço inicial da obra. Seu estilo é autêntico, criativo e centrado na experiência do leitor. É mestre em storytelling.",
            temperatura=0.8
        )

    def criar_estrutura_livro(self, tema, num_capitulos, conteudo_referencia=""):
        """
        Cria a estrutura básica do livro com títulos e resumos dos capítulos.

        Args:
            tema: Tema principal do livro
            num_capitulos: Número de capítulos do livro
            conteudo_referencia: Conteúdo extraído da URL de referência

        Returns:
            Dicionário com a estrutura do livro
        """
        print(f"🎬 {self.nome} está criando a estrutura do livro sobre {tema}...")

        referencia_text = ""
        if conteudo_referencia:
            referencia_text = f"\n\nUse como referência e inspiração o seguinte conteúdo (mas não copie diretamente):\n\n{conteudo_referencia[:5000]}"

        prompt = f"""
        Você é um {self.role}.

        {self.backstory}

        Seu objetivo é: {self.goal}

        Crie uma estrutura detalhada para um livro de {num_capitulos} capítulos sobre o tema: "{tema}".
        Levando em consideração o tom de voz do livro detalhado como {tom} e o contexto do livro detalhado como {contexto}.

        Para cada capítulo, forneça:
        1. Um título cativante e informativo, sendo que apenas a primeira letra do título estará em letra maiúscula
        2. Um resumo detalhado do conteúdo (3 parágrafos de até 70 palavras)
        3. 5 tópicos principais que serão abordados no capítulo

        O livro deve ter uma progressão lógica do básico ao avançado, cobrindo todos os aspectos importantes do tema.
        {referencia_text}

        Retorne sua resposta no seguinte formato para cada capítulo:

        # Capítulo X: [Título]

        ## Resumo
        [Resumo detalhado]

        ## Tópicos Principais
        - [Tópico 1]
        - [Tópico 2]
        - [Tópico 3]
        - [Tópico 4]
        - [Tópico 5 - opcional]
        - [Tópico 6 - opcional]
        """

        resposta = self._gerar_resposta(prompt)

        if resposta:
            print(f"✅ {self.nome} concluiu a estrutura do livro!")
            return resposta
        else:
            print(f"❌ {self.nome} não conseguiu criar a estrutura do livro.")
            return None

    def escrever_capitulo(self, estrutura_capitulo, numero_capitulo, conteudo_referencia=""):
        """
        Escreve o conteúdo completo de um capítulo com base na estrutura.

        Args:
            estrutura_capitulo: Estrutura detalhada do capítulo
            numero_capitulo: Número do capítulo
            conteudo_referencia: Conteúdo extraído da URL de referência

        Returns:
            Texto completo do capítulo
        """
        print(f"✍️ {self.nome} está escrevendo o Capítulo {numero_capitulo}...")

        referencia_text = ""
        if conteudo_referencia:
            referencia_text = f"\n\nUse como referência e inspiração o seguinte conteúdo (mas não copie diretamente):\n\n{conteudo_referencia[:5000]}"

        prompt = f"""
        Você é um {self.role}.

        {self.backstory}

        Seu objetivo é: {self.goal}

        Escreva o Capítulo {numero_capitulo} completo de um livro sobre {tema}, com base na estrutura detalhada abaixo:

        {estrutura_capitulo}

        Diretrizes para escrita:
        1. Mantenha um tom educacional mas conversacional, como se estivesse explicando para alguém interessado na área
        2. Inclua exemplos práticos, casos de uso e analogias para facilitar o entendimento
        3. Divida o texto em seções com subtítulos claros, seguindo a estrutura fornecida
        4. O texto deve ser aprofundado e abrangente, com aproximadamente 2300 palavras
        5. Inclua uma introdução contextual no início e uma conclusão que conecte com o próximo capítulo
        6. Onde relevante, mencione tendências atuais e perspectivas futuras
        7. Mantenha uma linguagem acessível, explicando termos técnicos quando introduzidos
        8. Leve em consideração o tom de voz do livro detalhado como {tom} e o contexto do livro detalhado como {contexto}
        {referencia_text}

        O texto deve ser completo, bem estruturado e pronto para revisão humana.
        """

        resposta = self._gerar_resposta(prompt)

        if resposta:
            print(f"✅ {self.nome} concluiu a escrita do Capítulo {numero_capitulo}!")
            return resposta
        else:
            print(f"❌ {self.nome} não conseguiu escrever o Capítulo {numero_capitulo}.")
            return None


class AgenteEditorLiterario(Agente):
    """
    Agente responsável por refinar e estruturar o conteúdo original do livro.
    """
    def __init__(self, modelo):
        super().__init__(
            modelo,
            "Agente Editor Literário",
            "Editor Literário",
            "Refinar e estruturar o conteúdo original do livro, elevando a qualidade literária e garantindo uma progressão fluida da narrativa.",
            "Você é um editor literário renomado, conhecido por transformar bons textos em obras-primas. Sua sensibilidade com estilo, ritmo e estrutura narrativa permite que você mantenha a voz do autor, enquanto aprimora a experiência de leitura. Você atua como mentor do texto, com atenção especial à coesão e clareza.",
            temperatura=0.6
        )

    def editar_capitulo(self, texto_capitulo, numero_capitulo, estrutura_capitulo):
        """
        Edita e melhora o conteúdo de um capítulo.

        Args:
            texto_capitulo: Texto completo do capítulo
            numero_capitulo: Número do capítulo
            estrutura_capitulo: Estrutura detalhada do capítulo

        Returns:
            Texto editado do capítulo
        """
        print(f"📝 {self.nome} está editando o Capítulo {numero_capitulo}...")

        prompt = f"""
        Você é um {self.role}.

        {self.backstory}

        Seu objetivo é: {self.goal}

        Edite e aprimore o Capítulo {numero_capitulo} sobre {tema} abaixo. Realize melhorias relacionadas a:

        1. Fluidez narrativa e ritmo do texto
        2. Transições entre parágrafos e seções
        3. Consistência de tom e voz do autor
        4. Impacto e engajamento do leitor
        5. Estrutura e progressão lógica das ideias
        6. Qualidade literária geral
        7. Eliminar redundâncias e partes menos relevantes

        Mantenha-se fiel ao tom de voz do livro descrito como: {tom}
        E ao contexto do livro descrito como: {contexto}

        Estrutura original do capítulo:
        {estrutura_capitulo}

        Texto atual do capítulo:
        {texto_capitulo}

        Faça as melhorias e refinamentos diretamente no texto. Retorne o capítulo completo editado.
        Não abrevie ou resuma o conteúdo - mantenha-o completo, apenas aprimorado.
        """

        resposta = self._gerar_resposta(prompt)

        if resposta:
            print(f"✅ {self.nome} concluiu a edição do Capítulo {numero_capitulo}!")
            return resposta
        else:
            print(f"❌ {self.nome} não conseguiu editar o Capítulo {numero_capitulo}.")
            return texto_capitulo  # Retorna o texto original se a edição falhar


class AgenteRevisor(Agente):
    """
    Agente responsável por revisar e corrigir o conteúdo escrito.
    """
    def __init__(self, modelo):
        super().__init__(
            modelo,
            "Agente Revisor",
            "Revisor Ortográfico e Gramatical",
            "Garantir que o texto final esteja impecável do ponto de vista ortográfico, gramatical e estilístico conforme as normas do português brasileiro.",
            "Você é um revisor com profundo conhecimento da gramática normativa da língua portuguesa, de acordo com o português brasileiro. Minucioso e exigente, seu trabalho é revisar cada linha em busca de erros de ortografia, pontuação, concordância e estilo, assegurando consistência e excelência linguística em toda a obra.",
            temperatura=0.4
        )

    def revisar_capitulo(self, texto_capitulo, numero_capitulo):
        """
        Revisa e corrige o conteúdo de um capítulo.

        Args:
            texto_capitulo: Texto completo do capítulo
            numero_capitulo: Número do capítulo

        Returns:
            Texto revisado do capítulo
        """
        print(f"🔍 {self.nome} está revisando o Capítulo {numero_capitulo}...")

        prompt = f"""
        Você é um {self.role}.

        {self.backstory}

        Seu objetivo é: {self.goal}

        Revise cuidadosamente o Capítulo {numero_capitulo} sobre {tema} abaixo. A revisão deve estar de acordo com o português brasileiro e verificar:

        1. Ortografia e acentuação
        2. Pontuação
        3. Concordância verbal e nominal
        4. Regência verbal e nominal
        5. Colocação pronominal
        6. Uso adequado dos tempos verbais
        7. Consistência terminológica
        8. Hifenização e uso de maiúsculas
        9. Uso correto do gerúndio e verbos
        10. Eliminação de estrangeirismos desnecessários

        Texto atual do capítulo:
        {texto_capitulo}

        Faça as correções necessárias diretamente no texto. Retorne o capítulo completo revisado e corrigido.
        Não abrevie ou resuma o conteúdo - mantenha-o completo, apenas corrigido.
        """

        resposta = self._gerar_resposta(prompt)

        if resposta:
            print(f"✅ {self.nome} concluiu a revisão do Capítulo {numero_capitulo}!")
            return resposta
        else:
            print(f"❌ {self.nome} não conseguiu revisar o Capítulo {numero_capitulo}.")
            return texto_capitulo  # Retorna o texto original se a revisão falhar


class GeradorDeLivro:
    """
    Coordena os agentes para a produção completa do livro.
    """
    def __init__(self, modelo):
        """
        Inicializa o gerador de livro com os três agentes.

        Args:
            modelo: Instância do modelo Gemini
        """
        self.autor = AgenteAutor(modelo)
        self.editor = AgenteEditorLiterario(modelo)
        self.revisor = AgenteRevisor(modelo)
        self.livro = {
            "titulo": "",
            "estrutura": "",
            "capitulos": {}
        }
        self.conteudo_referencia = ""

    def definir_referencia(self, url):
        """
        Define o conteúdo de referência para o livro a partir de uma URL.

        Args:
            url: URL da página web de referência
        """
        if url and url.strip() != "":
            self.conteudo_referencia = extrair_conteudo_url(url)

    def gerar_livro(self, tema=tema, num_capitulos=num_capitulos):
        """
        Gera um livro completo sobre o tema especificado.

        Args:
            tema: Tema principal do livro
            num_capitulos: Número de capítulos do livro

        Returns:
            Dicionário contendo o livro completo
        """
        print(f"📚 Iniciando a geração do livro sobre '{tema}' com {num_capitulos} capítulos...")

        # Etapa 1: Criar estrutura do livro
        self.livro["titulo"] = f"{tema}"
        self.livro["estrutura"] = self.autor.criar_estrutura_livro(tema, num_capitulos, self.conteudo_referencia)

        if not self.livro["estrutura"]:
            print("❌ Não foi possível criar a estrutura do livro. Processo abortado.")
            return None

        # Para cada capítulo
        for i in range(1, num_capitulos + 1):
            print(f"\n{'='*50}")
            print(f"📝 PROCESSANDO CAPÍTULO {i} DE {num_capitulos}")
            print(f"{'='*50}\n")

            # Etapa 2: Escrever o capítulo (o autor agora faz isso diretamente)
            conteudo_capitulo = self.autor.escrever_capitulo(self.livro["estrutura"], i, self.conteudo_referencia)

            if not conteudo_capitulo:
                print(f"⚠️ Não foi possível escrever o Capítulo {i}. Pulando para o próximo...")
                continue

            # Etapa 3: Editar o capítulo pelo editor literário
            conteudo_editado = self.editor.editar_capitulo(conteudo_capitulo, i, self.livro["estrutura"])

            if not conteudo_editado:
                conteudo_editado = conteudo_capitulo

            # Etapa 4: Revisar o capítulo
            conteudo_revisado = self.revisor.revisar_capitulo(conteudo_editado, i)

            # Armazenar o capítulo no livro
            self.livro["capitulos"][i] = {
                "conteudo_original": conteudo_capitulo,
                "conteudo_editado": conteudo_editado,
                "conteudo_final": conteudo_revisado
            }

            # Dar uma pausa para evitar limites de taxa da API
            print(f"😴 Pausa de 10 segundos para respeitar os limites da API...")
            time.sleep(10)

        print(f"\n{'='*50}")
        print(f"✅ LIVRO COMPLETO GERADO COM SUCESSO!")
        print(f"{'='*50}\n")

        return self.livro

    def salvar_livro_markdown(self, pasta_destino="livro_pronto"):
        """
        Salva cada capítulo do livro como arquivo markdown individual.

        Args:
            pasta_destino: Nome da pasta onde os arquivos serão salvos
        """
        # Criar a pasta se não existir
        if not os.path.exists(pasta_destino):
            os.makedirs(pasta_destino)
            print(f"📁 Pasta '{pasta_destino}' criada.")

        # Salvar a estrutura do livro
        with open(f"{pasta_destino}/00_estrutura_do_livro.md", "w", encoding="utf-8") as f:
            f.write(f"# {self.livro['titulo']}\n\n")
            f.write(self.livro["estrutura"])

        # Salvar cada capítulo
        for num_cap, capitulo in self.livro["capitulos"].items():
            # Extrair o título do capítulo da estrutura (simplificação)
            try:
                titulo_cap = f"Capítulo {num_cap}"
                linhas = capitulo["conteudo_final"].split("\n")
                for linha in linhas:
                    if linha.startswith("# ") or linha.startswith("## "):
                        titulo_cap = linha.strip("# ").strip()
                        break
            except:
                titulo_cap = f"Capítulo {num_cap}"

            # Salvar o conteúdo final do capítulo
            nome_arquivo = f"{pasta_destino}/{num_cap:02d}_{titulo_cap.replace(' ', '_').replace(':', '')}.md"
            with open(nome_arquivo, "w", encoding="utf-8") as f:
                f.write(capitulo["conteudo_final"])

            print(f"💾 Capítulo {num_cap} salvo como '{nome_arquivo}'")

    def exibir_capitulo(self, numero_capitulo):
        """
        Exibe um capítulo específico do livro no notebook usando Markdown.

        Args:
            numero_capitulo: Número do capítulo a ser exibido
        """
        if numero_capitulo not in self.livro["capitulos"]:
            print(f"❌ O Capítulo {numero_capitulo} não foi gerado ou não existe.")
            return

        capitulo = self.livro["capitulos"][numero_capitulo]
        display(Markdown(capitulo["conteudo_final"]))


# Função principal para executar o código
def livro():
    """
    Função principal que executa todo o processo de criação do livro.
    """
    print("🤖 Criando o livro...")
    print("==================================================")

    # Configurar API
    if not configurar_api():
        return

    # Criar modelo
    modelo = criar_modelo()
    if not modelo:
        return

    # Configurar e iniciar o gerador de livro
    gerador = GeradorDeLivro(modelo)

    # Definir o conteúdo de referência
    if url_referencia and url_referencia.strip() != "":
        gerador.definir_referencia(url_referencia)

    # Gerar o livro com os capítulos sobre o tema especificado
    livro = gerador.gerar_livro(tema=tema, num_capitulos=num_capitulos)

    if livro:
        # Salvar o livro em arquivos markdown
        gerador.salvar_livro_markdown()

        # Exibir um capítulo de exemplo
        print("\n📖 Exibindo o Capítulo 1 como exemplo:")
        gerador.exibir_capitulo(1)

        print("\n✅ Processo concluído com sucesso!")
        print("📚 O livro foi gerado e salvo em arquivos markdown.")
        print("🔍 Explore a pasta 'livro_pronto' para ver os arquivos gerados.")
    else:
        print("\n❌ Não foi possível gerar o livro completo.")


# Instruções para execução
if __name__ == "__main__":
    # Descomente a linha abaixo para executar o código automaticamente
    # livro()

    print("""
    ===== INSTRUÇÕES =====

    Para executar este código:

    1. O código já instala automaticamente os pacotes necessários
    2. Você precisa de uma chave API do Google AI Studio (Gemini API)
    3. Adicione sua chave API ao Google Colab:
       - Na barra lateral esquerda, clique no ícone de chave
       - Adicione uma nova chave secreta com nome "GEMINI_API_KEY" e o valor da sua chave API
    4. Alternativamente, você pode inserir a chave diretamente no código (veja a função configurar_api)
    5. Execute a função livro() para iniciar o processo de criação do livro

    Para iniciar, execute na linha de código abaixo:
    ```
    livro()
    ```
    """)


In [ ]:
start_time = time.time()
livro()
end_time = time.time()

elapsed_time = end_time - start_time
minutes, seconds = divmod(elapsed_time, 60)
centiseconds = (seconds - int(seconds)) * 100

print(f"Livro gerado em: {int(minutes)} minutos, {int(seconds)} segundos e {int(centiseconds)} centésimos")